In [ ]:
#ipcluster start -n=8

In [1]:
%pylab inline
import time
import numexpr as ne
import seaborn as sns
from ipywidgets import interact
from sklearn.decomposition import PCA
from numba import jit

Populating the interactive namespace from numpy and matplotlib


In [2]:
ne.set_num_threads(32)

8

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%gui qt

In [5]:
import sys
sys.path.append('../../..')

In [6]:
from spiketag.mvc import Sorter
from spiketag.base import ProbeFactory, MUA, bload, SPKTAG
from spiketag.utils import conf

# Test 32 liner probe

### pcie_32_25K_27/4/2017.bin 这个数据有点问题，bin是int32的，但是spk是int64的 

In [ ]:
nCh = 32
fs = 25000
probe = ProbeFactory.genLinearProbe(fs=fs, n_ch=nCh)

In [ ]:
# app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin',probe = probe, 
#               corr_cutoff=0.95, 
#               fetlen=6, 
#               fet_whiten=False,
#               fall_off_size=18, 
#               n_jobs=4)
app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin', 
             spktag_filename='/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin.probe')



In [ ]:
#app.model.spktag.tofile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin.probe')

In [ ]:
app.run()

# How to use line profiler to find bottomneck

In [ ]:
app.model.clu[30].selectlist

In [ ]:
app.model.clu[30].index

In [ ]:
# pip install line-profiler first.
%load_ext line_profiler

In [ ]:
%lprun -f app.view.trace_view.locate_and_highlight app.view.trace_view.locate_and_highlight(app.view.spk_view.selected_spk)

# Test 160 tetrode probe

In [7]:
nCh = 64
fs = 30000
tetrode = ProbeFactory.genTetrodeProbe(fs=fs, n_ch=64)

In [8]:
# app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_160_30K_bank0and1_25:5:2017.bin',probe = tetrode, 
#               corr_cutoff=0.95, 
#               fet_method='pca',
#               fetlen=6, 
#               fet_whiten=False,
#               fall_off_size=18, 
#               n_jobs=4)
app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_160_30K_bank0and1_25:5:2017.bin', spktag_filename='/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_160_30K_bank0and1_25:5:2017.bin.probe')

2017-07-04 21:13:12,142 - spiketag - INFO - load spktag file
2017-07-04 21:13:12,488 - spiketag - INFO - load mua data for wave view
2017-07-04 21:13:12,501 - spiketag - INFO - #############  load data  ###################
2017-07-04 21:13:12,502 - spiketag - INFO - /Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_160_30K_bank0and1_25:5:2017.bin loaded, it contains: 
2017-07-04 21:13:12,502 - spiketag - INFO - 18000000 * 64 points (4608000000 bytes) 
2017-07-04 21:13:12,503 - spiketag - INFO - 64 channels with sampling rate of 30000.0000 
2017-07-04 21:13:12,503 - spiketag - INFO - 600.000 secs (10.000 mins) of data
2017-07-04 21:13:12,504 - spiketag - INFO - #############################################


In [ ]:
#app.model.spktag.tofile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_160_30K_bank0and1_25:5:2017.bin.probe')

In [9]:
app.run()

# 问题1：key不存在的时候, 弹出提示没有clu

# 问题2：wave_view.set_data有问题

In [ ]:
app.view.wave_view.set_data(17,app.model.clu[17])

In [ ]:
app.view.spk_view.set_data(app.model.spk[57], app.model.clu[57])

In [ ]:
app.view.spk_view.depth.shape

In [ ]:
app.view.spk_view.color.shape

In [ ]:
app.view.spk_view.signal_index.shape

In [ ]:
app.view.spk_view.box_index.shape

In [ ]:
app.model.spktag.probe.get_chs(42)

In [ ]:
plot(app.model.spk[57][10,:,:3])

In [ ]:
app.view.spk_view.color.depth

In [ ]:
app.view.wave_view.clu.membership.shape

In [ ]:
app.view.amplitude_view._clu.membership.shape

### Detect peak and output pivotal file

In [ ]:
bf = bload(nCh=64, fs=25000)

In [ ]:
bf.load('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/tt16bit_160.bin', dtype='int16')

In [ ]:
spks = bf.detect_spks()

In [ ]:
data = bf.asarray()

In [ ]:
from ipywidgets import interact

In [ ]:
@interact(x=(1000,55752704,100), y=(0,63,1))
def plot_wave(x,y):
    plot(data[x-1000:x,y])

In [ ]:
from spiketag.view import wave_view

In [ ]:
data = bf.npmm.reshape(-1,64).astype(np.float32)

In [ ]:
wv = wave_view(data=data)

In [ ]:
wv.show()

In [ ]:
raw_data = np.fromfile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/tt16bit_160.bin', dtype=np.int16) 

In [ ]:
filted_data = raw_data.reshape(-1,64)[:11400000,:]

In [ ]:
filted_data.astype(np.int32).tofile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/tt16bit_160_filted.bin')

In [ ]:
bf = bload(nCh=32, fs=25000)

In [ ]:
bf.load('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25K_27:4:2017.bin', dtype='int16')

In [ ]:
data = bf.npmm.reshape(-1,64).astype(np.float32)

In [ ]:
spks = bf.detect_spks(0.003)

In [ ]:
@interact(x=(1000,11400000,100), y=(0,63,1))
def plot_wave(x,y):
    plot(data[x-1000:x,y])

In [ ]:
wv2 = wave_view(data=data, spks=spks)

In [ ]:
wv2.show()

In [ ]:
spks.dtype

In [ ]:
spks.astype(np.int32).tofile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/tt16bit_160_filted.bin.spk')

In [ ]:
spks

In [ ]:
mua = MUA('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/tt16bit_160_filted.bin', tetrode)

In [ ]:
mua.pivotal_pos

In [ ]:
spks

In [ ]:
np.fromfile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/tt16bit_160_filted.bin.spk', dtype='<i4').reshape(-1,2).T

In [ ]:
np.fromfile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/tt16bit_160_filted.bin.spk', dtype='<i4')

In [ ]:
np.fromfile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin.spk', dtype='<i4')

In [ ]:
np.column_stack((spks[0],spks[1])).ravel().astype(np.int32).tofile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/tt16bit_160_filted.bin.spk')

In [ ]:
spks = np.fromfile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/spks_32_25K_27:4:2017', dtype='int64').reshape(2,-1)

In [ ]:
np.column_stack((spks[0],spks[1])).ravel().astype(np.int32).tofile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25K_27:4:2017.bin.spk')

In [ ]:
nCh=32
fs=25000
probe = ProbeFactory.genLinearProbe(fs=fs, n_ch=nCh)
mua = MUA('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25K_27:4:2017.bin', probe)

In [ ]:
mua.remove_high_corr_noise(corr_cutoff=0.9)

In [ ]:
spk = mua.tospk()

In [ ]:
mua.pivotal_pos

In [ ]:
np.unique(mua.pivotal_pos[1])

In [ ]:
spks = np.fromfile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25K_27:4:2017.bin.spk', dtype='int64').reshape(2,-1)

In [ ]:
spks


In [ ]:
spks = np.fromfile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/spks_32_25K_27:4:2017', dtype='i8').reshape(2,-1)

In [ ]:
spks

In [ ]:
bf = bload(fs=fs, nCh=nCh)

In [ ]:
bf.load('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25K_27:4:2017.bin')

In [ ]:
data = bf.asarray()

In [ ]:
data.shape

In [ ]:
plot(data[:1000,1])

In [ ]:
bf.load('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25K_27:4:2017.bin', dtype='int64')

In [ ]:
data = bf.asarray()

In [ ]:
data.shape

In [ ]:
plot(data[:1000,1])

In [ ]:
s1 = np.fromfile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25K_27:4:2017.bin.spk', dtype='i4').reshape(2,-1)

In [ ]:
s2 = np.fromfile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25K_27:4:2017_int64.bin.spk', dtype='i8')

In [ ]:
np.equal(s1,s2).all()

In [ ]:
s1.tofile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/int32')

In [ ]:
s1

In [ ]:
result = np.column_stack((s1[0],s1[1]))

In [ ]:
result

In [ ]:
result.ravel().tofile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/int32')

In [ ]:
from phy.gui.qt import _prompt, _show_box

In [ ]:
b = _prompt("Channel 1 has no spikes"
            "", buttons=['close'],title='Waring')

In [ ]:
r = _show_box(b)

In [ ]:
import vispy


In [ ]:
vispy.test()

In [ ]:
print(vispy.sys_info())